In [13]:

%env ENV_PROFILE=dev

import logging
from datetime import datetime

from src.common.candle_client import CandleInterval
from src.constants import TimeZone
from src.container import ApplicationContainer

logging.basicConfig(level=logging.DEBUG, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s")

di_container = ApplicationContainer()  # DI Container 초기화 및 자동 와이어링

upbit_api = di_container.upbit_api()
candle_service = di_container.candle_service()
ticker_repository = di_container.ticker_repository()
candle_query_service = di_container.candle_query_service()

hantu_domestic_api = di_container.hantu_domestic_api()
# hantu_overseas_api = HantuOverseasAPI(HantuConfig())  # type: ignore

# v_hantu_domestic_api = HantuDomesticAPI(HantuConfig(), AccountType.VIRTUAL)  # type: ignore
# v_hantu_overseas_api = HantuOverseasAPI(HantuConfig(), AccountType.VIRTUAL)  # type: ignore

# auto_data_google_sheet_client = GoogleSheetClient(GoogleSheetConfig(), "auto_data")
# kimp_data_google_sheet_client = GoogleSheetClient(GoogleSheetConfig(), "kimp_data")

# bithumb_config = BithumbConfig()

# upbit_api.get_candles(market=KRW_BTC, interval=UpbitCandleInterval.MINUTE_1, count=2)

ticker = ticker_repository.find_by_ticker("QQQ")
# candle_service.collect_minute1_candles(
#     ticker=ticker,
#     mode=CollectMode.FULL,
#     # start=datetime(2026, 1, 21, 0, 0, tzinfo=TimeZone.SEOUL.tz),
#     # to=datetime(2026, 1, 21, 9, 0, tzinfo=TimeZone.SEOUL.tz)
# )
candle_query_service.get_candles(
    ticker,
    interval=CandleInterval.MINUTE_1,
    count=1000,
    end_time=datetime(2025, 12, 15, 19, 59, tzinfo=TimeZone.NEW_YORK.tz)
)

env: ENV_PROFILE=dev


2026-01-23 10:48:36,181 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): openapi.koreainvestment.com:9443
2026-01-23 10:48:36,316 - urllib3.connectionpool - DEBUG - https://openapi.koreainvestment.com:9443 "GET /uapi/overseas-price/v1/quotations/inquire-time-itemchartprice?AUTH=&EXCD=NAS&SYMB=QQQ&NMIN=1&PINC=1&NEXT=1&NREC=120&FILL=&KEYB=20251215195900 HTTP/1.1" 200 222
2026-01-23 10:48:36,316 - src.hantu.overseas_api - DEBUG - No more data available (empty response). Total: 0 candles


,timestamp,local_time,open,high,low,close,volume
